In [118]:
import logging 
import pandas as pd
import boto3
import json 
from dataclasses import dataclass, field
from pydantic import validate_arguments
from ics import Calendar, Event
from typing import Optional, List, Dict
from datetime import datetime

In [36]:
def convert_to_12_hr(time_str: str) -> str:
    "Lambda function to convert 24 hour time to 12 hour time"
    time_as_24_hr = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S %z").time()
    time_as_12_hr = time_as_24_hr.strftime("%-I:%M %p")
    return time_as_12_hr

In [129]:
@dataclass 
class AppleHealthEvent(Event):
    """
    An event derived from Apple Health data
    For usage within .ics format 
    """
    date: datetime.date
    description: str 
    title: str
            
    @property 
    def event(self):
        all_day_date = f"{self.date} 00:00:00"
        e = Event()
        e.name = self.title
        e.description = self.description
        e.begin = all_day_date
        e.end = all_day_date
        e.make_all_day()

        return e

In [130]:
@validate_arguments
@dataclass
class AppleHealthData:
    """
    A dataclass to hold all the data from Apple Health
    Parsing from AWS API Gateway and S3
    """
    date: str
    date_updated: str
    name: str
    qty: float
    units: str
    source: Optional[str] = None
    
    def __post_init__(self): 
        self.date = datetime.strptime(self.date, '%Y-%m-%d %H:%M:%S %z')
        self.date_updated = datetime.strptime(self.date_updated, '%Y-%m-%d %H:%M:%S.%f')
    

@validate_arguments
@dataclass
class Time: 
    "A basic time object in hours"
    time: Optional[float] = field(default=0)
    timeInMinutes: Optional[float] = field(default=0)
        
    def __post_init__(self): 
        if self.timeInMinutes:
            self.time = self.timeInMinutes / 60
            
        
    @property 
    def hours(self) -> float: 
        hours = int(self.time)
        return hours 
    
    @property
    def minutes(self) -> str:
        minutes = int((self.time - self.hours) * 60)
        return minutes
    
    @property
    def title(self) -> str:
        title = f"{self.hours}h {self.minutes}m"
        return title

@validate_arguments
@dataclass 
class Food: 
    "A basic food object"
    carbohydrates: float 
    protein: float 
    total_fat: float 
    fiber: float 
    calories_burnt: Optional[float] = field(default=0)

    def __post_init__(self): 
        # rename objects for easier usage 
        self.carb = self.carbohydrates
        self.fat = self.total_fat
        self.calories_burnt = round(self.calories_burnt / 4,2)
    
    @property 
    def calories_ate(self) -> float: 
        calories = (self.carb + self.protein) * 4 + (self.fat) * 9
        return calories
    
    @property 
    def macros(self) -> str: 
        return f"{self.carb:.0f}C, {self.protein:.0f}P, {self.fat:.0f}F"

    @property 
    def title(self) -> str: 
        title = f"🔥 {self.calories_ate:.0f} cals ({self.macros})" 
        return title 
    
    @property 
    def description(self) -> str: 
        description = f"""
        🔥 {self.calories_burnt:.0f} kcal
        🍽️ {self.calories_ate:.0f} kcal
        🥞 {self.macros} 
        🍇 {self.fiber:.0f} g
        """
        return description

@validate_arguments
@dataclass
class Activity: 
    "A basic activity for activity and mindfulness"
    apple_exercise_time: float
    mindful_minutes: float = None
    
    def __post_init__(self): 
        # rename objects for easier usage
        self.apple_exercise_time = Time(timeInMinutes=self.apple_exercise_time)
        self.mindful_minutes = Time(timeInMinutes=self.mindful_minutes)
    
    @property 
    def activity_description(self) -> str: 
        a_description = f"🚴‍♂️ Activity: {self.apple_exercise_time.title} active" 
        return a_description

    @property 
    def mindful_description(self) -> str:
        m_description = f"🧘 Mindful: {self.mindful_minutes.title} mindful"
        return m_description
     
    @property 
    def description(self) -> str: 
        description = f"""
        {self.activity_description}
        {self.mindful_description}
        """
        return description
    
    @property 
    def mindful_title(self) -> str: 
        title = f"🧠 {self.mindful_minutes.minutes} mins "
        return title
    
    @property 
    def activity_title(self) -> str: 
        "Create blocks of 1 hour increments of activity minutes"
        block = str(self.apple_exercise_time.hours)
        title = f"🚴‍♂️ {block}"
        return title 
  
    @property 
    def title(self) -> str: 
        title = f"{self.mindful_title} | {self.activity_title}"
        return title 
   

@validate_arguments
@dataclass
class Sleep:
    "A basic sleep object"
    asleep: float
    inBed: float
    inBedStartTime: str

    def __post_init__(self): 
        # rename objects for easier usage
        self.time_asleep = Time(time=self.asleep)
        self.time_in_bed = Time(self.inBed)
        self.in_bed_time = convert_to_12_hr(self.inBedStartTime)
    
    @property 
    def efficiency(self) -> float: 
        efficiency = self.time_asleep.time / self.time_in_bed.time * 100
        return efficiency
    
    @property
    def efficiency_title(self) -> str: 
        efficient = f"{self.efficiency:.0f}%"
        efficiency_title = f"🛏️ {efficient}"
        return efficiency_title
    
    @property 
    def title(self) -> str: 
        title = f"💤 {self.time_asleep.title} ({self.in_bed_time})"
        print(title)
        return title 
    
    @property
    def description(self) -> str: 
        s_description = f"""
        💤 Time asleep: {self.time_asleep.title}
        🛏️ Time in bed: {self.time_in_bed.title}
        🧮 Efficiency: {self.efficiency_title}
        """
        return s_description


In [107]:
def get_s3_keys(bucket):
    """Get a list of keys in an S3 bucket."""
    keys = []
    personal = boto3.Session(profile_name='personal')
    s3 = personal.client('s3')
    resp = s3.list_objects(Bucket=bucket)
    for obj in resp['Contents']:
        keys.append(obj['Key'])
    return keys

In [108]:
event_objects_mapping = json.loads(open('flask/mapping.json').read())

In [125]:
# Functions
def collect_event_stats(
    stats_df: pd.DataFrame, 
    column_names: List[str]
)-> pd.DataFrame: 
    """
    Extract health data from stats dataframe in the format 
    [['name', 'qty']] where: 
    - name: name of the health data
    - qty: value (in respective unit) of the health data
    
    Output: 
        - DataFrame of health data in the format [['name', 'qty']]
    """
    filtered_stats = stats_df[stats_df['name'].isin(column_names)]
    event_type_stats  = filtered_stats[['name', 'qty']]
    
    return event_type_stats

def create_day_events(
    stats: pd.DataFrame, 
    event_date: str,
    object_mapping: dict 
) -> List[Event]: 
    """
    Iterate through different event types (food / activity / sleep)
    and generate events to add to the daily calendar only if event exists
    """
    logging.info(f"Creating {types} event")
    
    day_events = []
    
    for types, col_names in object_mapping.items(): 
        logging.info(f"Creating {types} event")
        # collect object name and arguments
        # dynamically create event type objects 
        dataclass_name = types
        dataclass_obj = globals()[dataclass_name]
        
        dataclass_obj_stats = collect_event_stats(
            stats_df=stats, 
            column_names=col_names
        )
        
        obj_args = dict(dataclass_obj_stats.values)
        
        if obj_args: 
            obj = dataclass_obj(**obj_args)
            e = AppleHealthEvent(
                date = event_date,
                title = obj.title, 
                description = obj.description
            ).event
            day_events.append(e)
            
    return day_events

In [54]:
get_s3_keys(bucket='ntonthat-apple-health-data')

['parquets/2023-01-18T09:39:59.parquet',
 'syncs/2023-01-18T09:39:59.479199.json',
 'workouts/2023-01-18T09:39:59.828797.json']

In [12]:
def load_json_from_s3(bucket, key):
    personal = boto3.Session(profile_name='personal')
    s3 = personal.resource('s3')
    
    obj = s3.Object(bucket, key)
    body = obj.get()['Body']
    body = body.read().decode('utf-8')
    return body

In [147]:
def read_s3_trigger(trigger_event): 
    logging.info('Reading s3 event trigger')
    s3_event = json.loads(open(f'{trigger_event}', 'r').read())
    
    s3_key = s3_event.get("Records")[0].get('s3').get('object').get('key')
    s3_bucket = s3_event.get("Records")[0].get('s3').get('bucket').get('name')

    return s3_bucket, s3_key

In [157]:
def create_parquets(event): 
    s3 = boto3.resource('s3')
    personal = boto3.Session(profile_name='personal')
    s3 = personal.resource('s3')
    bucket, key = read_s3_trigger(event)

    # convert contents to native python string
    json_data = load_json_from_s3(bucket, key)
    
    source_data = []
    
    for line in json_data.splitlines(): 
        source_data.append(json.loads(line))
        
    logging.info('Converting to dataframe')
    df = pd.DataFrame.from_records(source_data)
    
    # force conversion types
    df['qty'] = df['qty'].astype(str)
    df['date'] = pd.to_datetime(df['date']).dt.date
    
    logging.info('Converting to parquet')
    
    # write to parquet
    file_name = key.split('/')[-1].split('.')[0]
    parquet_file_name = f'{file_name}.parquet'
    df.to_parquet(f'{parquet_file_name}')
    
    s3.meta.client.upload_file(f'{parquet_file_name}', bucket, f'parquets/{parquet_file_name}')
    
    # convert files into calendar 

    c = Calendar()
    available_dates = df['date'].unique()

    for date in available_dates:
        daily_stats = df[df['date'] == date]   
        daily_calendar = create_day_events(
            stats=daily_stats,
            event_date=date,
            object_mapping=event_objects_mapping
        )
        for event in daily_calendar:
            c.events.add(event)
    
    calendar_file_name = "outputs/apple-health-calendar.ics"
    # ics_file_name = 'outputs/apple-health.ics'
    with open(calendar_file_name, 'w') as f:
        logging.info("Writing calendar to file locally")
        f.write(c.serialize())
        f.close()
        
        
    s3.meta.client.upload_file(f"{calendar_file_name}", bucket, calendar_file_name)
    return df

In [155]:
from  pathlib import Path
import os 
parquet_path = Path('outputs/parquets')

# parquets = [os.path.join(parquet_path, f) for f in os.listdir(parquet_path) if f.endswith('.parquet')]

df = pd.read_parquet(parquet_path)
    

In [156]:
create_parquets(event='source/s3-trigger-event.json')

FileNotFoundError: [Errno 2] No such file or directory: '2023-01-26T06:11:46.ics'

/Users/ntonthat/Library/Caches/pypoetry/virtualenvs/apple-health-calendar-FpPaTjHZ-py3.11/lib/python3.11/site-packages/ics/component.py:85: FutureWarning: Behaviour of str(Component) will change in version 0.9 to only return a short description, NOT the ics representation. Use the explicit Component.serialize() to get the ics representation.
  warnings.warn(


In [14]:
df = create_parquets('flask/healthlake/s3-trigger-event.json')